# UKWA Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [wsw-results](https://github.com/Logiqx/wsw-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

### Summary

Adhoc script to determine the correctness of UKWA flags in the current year

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import csv

from datetime import datetime

from name import Name
from fuzzy import FuzzyMatch

In [2]:
PROJDIR = os.path.realpath(os.path.join(sys.path[0], '..'))

## Process Names

Process all names

In [3]:
def loadEntrants(year):

    csvPath = os.path.join(PROJDIR, 'events', str(year), 'config', 'entrants.csv')

    entrants = []
    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for entrant in csvReader:
            entrants.append(entrant)

    return entrants

In [4]:
def loadMembers(year):

    csvPath = os.path.join(PROJDIR, 'events', str(year), 'config', 'ukwa.csv')

    entrants = []
    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.reader(f)
        for entrant in csvReader:
            name = Name(entrant[0])
            entrants.append(name)
            
    return entrants

In [5]:
def checkEntrants(entrants, memberNames):

    fuzzyMatch = FuzzyMatch()
    
    yes = []
    no = []

    fuzzy = 0
    for entrant in entrants:
        entrantName = Name(entrant['First Name'] + ' ' + entrant['Family Name'])
        
        match = None
        for memberName in memberNames:
            if fuzzyMatch.matchNameObjects(entrantName, memberName):
                match = memberName
                
                if entrantName.name != memberName.name:
                    print('Fuzzy match detected {} vs {}'.format(entrantName.name, memberName.name))
                    fuzzy += 1
                    
        if  entrant['UKWA Member'].upper() == 'N' and match and entrant['Craft Type'] in ['Sailboard', 'Wingboard']:
            yes.append('{} ({}) - {}'.format(entrantName.name, entrant['ID'], entrant['Craft Type']))
            
        if entrant['UKWA Member'].upper() == 'Y' and not (match and entrant['Craft Type'] in ['Sailboard', 'Wingboard']):
            no.append('{} ({}) - {}'.format(entrantName.name, entrant['ID'], entrant['Craft Type']))

    if fuzzy:
        print()

    if yes:
        print('People who have not marked themselves as UKWA but are a member:')
        for entrant in yes:
            print(entrant)
        print()

    if no:
        print('People who have marked themselves as UKWA but are NOT a member:')
        for entrant in no:
            print(entrant)
        print()

In [6]:
def main():
    year = datetime.today().year
    
    entrants = loadEntrants(year)
    memberNames = loadMembers(year)
    
    checkEntrants(entrants, memberNames)
    

if __name__ == '__main__':
    main()

Fuzzy match detected Kev Greenslade vs Kevin Greenslade
Fuzzy match detected Nic Dodd vs Nicholas Dodd
Fuzzy match detected Scotty Stallman vs Scott Stallman
Fuzzy match detected Thomas Harrison vs thomas harrison
Fuzzy match detected Will Trossell vs William Trossell

People who have marked themselves as UKWA but are NOT a member:
Finlay Chandler (28) - Wingboard
Peter Martin (87) - Kiteboard



## All Done!